In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms

class ClassicCNN(nn.Module):
    def __init__(self):
        super(ClassicCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1 = None
        self.fc2 = nn.Linear(128, 10)
        self.flatten_dim=None

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.bn1(x)
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        if self.flatten_dim is None:
            print("Dimensione dopo convoluzione:", x.shape)
            self.flatten_dim = x.shape[1] * x.shape[2] * x.shape[3]  # Calcoliamo la dimensione corretta
            self.fc1 = nn.Linear(self.flatten_dim, 128).to(x.device)  # Inizializziamo `fc1` dinamicamente

        x = x.reshape(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = ClassicCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')


cuda


RuntimeError: running_mean should contain 16 elements not 8

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms
import numpy as np


def to_frequency_domain(x):
    return torch.fft.fft2(x)

def to_time_domain(x):
    return torch.fft.ifft2(x).real

class FrequencyConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(FrequencyConv, self).__init__()
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels
        self.weights = nn.Parameter(torch.randn(1, out_channels, in_channels, kernel_size, kernel_size, dtype=torch.cfloat))

    def forward(self, x):
        batch_size, _, H, W = x.shape
        
        kernel_padded = torch.zeros((batch_size, self.out_channels, self.in_channels, H, W), dtype=torch.cfloat, device=x.device)
        kernel_padded[:, :, :, :self.kernel_size, :self.kernel_size] = self.weights 
        kernel_freq = to_frequency_domain(kernel_padded)
        
        out_freq = x.unsqueeze(1) * kernel_freq
        return out_freq.sum(dim=2)
    
class CEMNet(nn.Module):
    def __init__(self):
        super(CEMNet, self).__init__()
        self.conv1 = FrequencyConv(1, 8, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = FrequencyConv(8, 16, 3)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = None
        self.fc2 = nn.Linear(128, 10)
        self.flatten_dim=None

        
    
    def forward(self, x):
        x = self.conv1(x)
        x = torch.abs(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = torch.abs(x)
        x = self.bn2(x)
        if self.flatten_dim is None:
            print("Dimensione dopo convoluzione:", x.shape)
            self.flatten_dim = x.shape[1] * x.shape[2] * x.shape[3]  # Calcoliamo la dimensione corretta
            self.fc1 = nn.Linear(self.flatten_dim, 128).to(x.device)  # Inizializziamo `fc1` dinamicamente

        x = x.reshape(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.fc2(x)
        return x




saved_data = torch.load('./transformed_mnist.pt',weights_only=False)

train_images = saved_data['train_images']
train_labels = saved_data['train_labels']
test_images = saved_data['test_images']
test_labels = saved_data['test_labels']

trainset = torch.utils.data.TensorDataset(train_images, train_labels)
testset = torch.utils.data.TensorDataset(test_images, test_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print("Dataset loaded successfully.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CEMNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')

Dataset loaded successfully.
cuda
Dimensione dopo convoluzione: torch.Size([64, 16, 28, 28])
Epoca [1/5], Loss: 0.6981, Accuracy: 81.49%
Epoca [2/5], Loss: 0.3786, Accuracy: 89.07%
Epoca [3/5], Loss: 0.3485, Accuracy: 89.95%
Epoca [4/5], Loss: 0.3351, Accuracy: 90.36%
Epoca [5/5], Loss: 0.3271, Accuracy: 90.56%
Accuracy sui test: 91.00%


**CIFRA10**

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms

class ClassicCNN(nn.Module):
    def __init__(self):
        super(ClassicCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.flatten_dim = None  # Per salvare la dimensione dinamica
        self.fc1 = None  # Sarà inizializzato dinamicamente
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.bn1(x)
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        if self.flatten_dim is None:
            print("Dimensione dopo convoluzione:", x.shape)
            self.flatten_dim = x.shape[1] * x.shape[2] * x.shape[3]  # Calcoliamo la dimensione corretta
            self.fc1 = nn.Linear(self.flatten_dim, 128).to(x.device)  # Inizializziamo `fc1` dinamicamente

        x = x.reshape(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = ClassicCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
cuda
Dimensione dopo convoluzione: torch.Size([64, 32, 32, 32])
Epoca [1/5], Loss: 1.5626, Accuracy: 43.92%
Epoca [2/5], Loss: 1.3127, Accuracy: 53.23%
Epoca [3/5], Loss: 1.2274, Accuracy: 56.30%
Epoca [4/5], Loss: 1.1716, Accuracy: 58.37%
Epoca [5/5], Loss: 1.1336, Accuracy: 59.82%
Accuracy sui test: 57.97%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision
import torchvision.transforms as transforms


def to_frequency_domain(x):
    return torch.fft.fft2(x)

def to_time_domain(x):
    return torch.fft.ifft2(x).real

class FrequencyConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(FrequencyConv, self).__init__()
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels
        # Creiamo un peso per ogni canale di ingresso
        self.weights = nn.Parameter(torch.randn(1, out_channels, in_channels, kernel_size, kernel_size, dtype=torch.cfloat))

    def forward(self, x):
        batch_size, _, H, W = x.shape
        
        # Iniziamo con un kernel zero-padded
        kernel_padded = torch.zeros((batch_size, self.out_channels, self.in_channels, H, W), dtype=torch.cfloat, device=x.device)
        
        # Copiamo i pesi per ogni canale di ingresso
        kernel_padded[:, :, :, :self.kernel_size, :self.kernel_size] = self.weights
        
        # Trasformiamo il kernel in dominio di frequenza
        kernel_freq = to_frequency_domain(kernel_padded)
        
        # Calcoliamo la convoluzione nel dominio delle frequenze
        out_freq = x.unsqueeze(1) * kernel_freq
        return out_freq.sum(dim=2)
    
class CEMNet(nn.Module):
    def __init__(self):
        super(CEMNet, self).__init__()
        self.conv1 = FrequencyConv(3, 16, 5)  # Da 3 a 8 canali
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = FrequencyConv(16, 32, 5)  # Da 8 a 16 canali
        self.bn2 = nn.BatchNorm2d(32)
        self.flatten_dim = None  # Per salvare la dimensione dinamica
        self.fc1 = None  # Sarà inizializzato dinamicamente
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)  # Output: [batch, 8, H, W]
        x = torch.abs(x)
        x=torch.relu(x)

        x = self.bn1(x)
        x = self.conv2(x)  # Output: [batch, 16, H, W]
        x = torch.abs(x)
        x=torch.relu(x)

        x = self.bn2(x)
        # Debug: stampiamo la dimensione dopo la convoluzione
        if self.flatten_dim is None:
            print("Dimensione dopo convoluzione:", x.shape)
            self.flatten_dim = x.shape[1] * x.shape[2] * x.shape[3]  # Calcoliamo la dimensione corretta
            self.fc1 = nn.Linear(self.flatten_dim, 128).to(x.device)  # Inizializziamo `fc1` dinamicamente

        x = x.reshape(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.fc2(x)
        return x


# Caricamento dataset HSV
saved_data = torch.load('./transformed_cifar10_hsv.pt', weights_only=False)

train_images = saved_data['train_images']  # [N, 3, 32, 32]
train_labels = saved_data['train_labels']
test_images = saved_data['test_images']
test_labels = saved_data['test_labels']

trainset = torch.utils.data.TensorDataset(train_images, train_labels)
testset = torch.utils.data.TensorDataset(test_images, test_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print("Dataset loaded successfully.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CEMNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

# Valutazione sul test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')


Dataset loaded successfully.
cuda
Dimensione dopo convoluzione: torch.Size([64, 12, 36, 36])
Epoca [1/5], Loss: 1.9239, Accuracy: 30.71%
Epoca [2/5], Loss: 1.7182, Accuracy: 39.05%
Epoca [3/5], Loss: 1.6502, Accuracy: 41.54%
Epoca [4/5], Loss: 1.6141, Accuracy: 43.06%
Epoca [5/5], Loss: 1.5907, Accuracy: 43.82%
Accuracy sui test: 43.59%


**Cifra10 RGB**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.fft
import torchvision.transforms as transforms


def to_frequency_domain(x):
    return torch.fft.fft2(x)

def to_time_domain(x):
    return torch.fft.ifft2(x).real

class FrequencyConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(FrequencyConv, self).__init__()
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels
        # Creiamo un peso per ogni canale di ingresso
        self.weights = nn.Parameter(torch.randn(1, out_channels, in_channels, kernel_size, kernel_size, dtype=torch.cfloat))

    def forward(self, x):
        batch_size, _, H, W = x.shape
        
        # Iniziamo con un kernel zero-padded
        kernel_padded = torch.zeros((batch_size, self.out_channels, self.in_channels, H, W), dtype=torch.cfloat, device=x.device)
        
        # Copiamo i pesi per ogni canale di ingresso
        kernel_padded[:, :, :, :self.kernel_size, :self.kernel_size] = self.weights
        
        # Trasformiamo il kernel in dominio di frequenza
        kernel_freq = to_frequency_domain(kernel_padded)
        
        # Calcoliamo la convoluzione nel dominio delle frequenze
        out_freq = x.unsqueeze(1) * kernel_freq
        return out_freq.sum(dim=2)
    
class CEMNet(nn.Module):
    def __init__(self):
        super(CEMNet, self).__init__()
        self.conv1 = FrequencyConv(3, 16, 5)  # Da 3 a 8 canali
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = FrequencyConv(16, 32, 5)  # Da 8 a 16 canali
        self.bn2 = nn.BatchNorm2d(32)
        self.flatten_dim = None  # Per salvare la dimensione dinamica
        self.fc1 = None  # Sarà inizializzato dinamicament       
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)  # Output: [batch, 8, H, W]
        x = torch.abs(x)
        x=torch.relu(x)

        x = self.bn1(x)
        x = self.conv2(x)  # Output: [batch, 16, H, W]
        x = torch.abs(x)
        x=torch.relu(x)

        x = self.bn2(x)
        # Debug: stampiamo la dimensione dopo la convoluzione
        if self.flatten_dim is None:
            print("Dimensione dopo convoluzione:", x.shape)
            self.flatten_dim = x.shape[1] * x.shape[2] * x.shape[3]  # Calcoliamo la dimensione corretta
            self.fc1 = nn.Linear(self.flatten_dim, 128).to(x.device)  # Inizializziamo `fc1` dinamicamente

        x = x.reshape(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.fc2(x)
        return x


# Caricamento dataset HSV
saved_data = torch.load('./transformed_cifar10.pt', weights_only=False)

train_images = saved_data['train_images']  # [N, 3, 32, 32]
train_labels = saved_data['train_labels']
test_images = saved_data['test_images']
test_labels = saved_data['test_labels']

trainset = torch.utils.data.TensorDataset(train_images, train_labels)
testset = torch.utils.data.TensorDataset(test_images, test_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print("Dataset loaded successfully.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CEMNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    total = 0
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoca [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

# Valutazione sul test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Accuracy sui test: {test_accuracy:.2f}%')


Dataset loaded successfully.
cuda
Dimensione dopo convoluzione: torch.Size([64, 32, 36, 36])


KeyboardInterrupt: 